# 🚀 Day 4: Error Handling & Debugging

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sdodlapa/cuda-lab/blob/main/learning-path/week-01/day-4-error-handling.ipynb)

> **Note:** If running on Google Colab, go to `Runtime → Change runtime type → T4 GPU` before starting!

**Structure:**
- **Part 1: CUDA C++ (Primary)** - Native CUDA error handling
- **Part 2: Python/Numba (Optional)** - Alternative for rapid prototyping

## Part 1: CUDA C++ (Primary)

### GPU Check

In [ ]:
!nvcc --version
!nvidia-smi --query-gpu=name,compute_cap --format=csv

### Why Error Handling Matters

CUDA operations are **asynchronous**. Errors may not appear until later!

```cpp
kernel<<<grid, block>>>(...);  // Launches, returns immediately
// ... other code ...
cudaDeviceSynchronize();       // Error might appear HERE!
```

### The CHECK_CUDA Macro

The standard pattern for CUDA error checking:

In [ ]:
%%writefile error_checking.cu
/**
 * CUDA Error Checking Patterns
 * 
 * Essential techniques for debugging CUDA code.
 */

#include <stdio.h>
#include <cuda_runtime.h>

// =====================================================
// THE ESSENTIAL ERROR CHECKING MACRO
// =====================================================
#define CHECK_CUDA(call)                                                    \
    do {                                                                    \
        cudaError_t err = call;                                            \
        if (err != cudaSuccess) {                                          \
            fprintf(stderr, "CUDA Error at %s:%d - %s\n",                  \
                    __FILE__, __LINE__, cudaGetErrorString(err));          \
            exit(EXIT_FAILURE);                                            \
        }                                                                  \
    } while (0)

// Check for errors AFTER kernel launch (kernels don't return cudaError_t)
#define CHECK_KERNEL()                                                      \
    do {                                                                    \
        cudaError_t err = cudaGetLastError();                              \
        if (err != cudaSuccess) {                                          \
            fprintf(stderr, "Kernel Error at %s:%d - %s\n",                \
                    __FILE__, __LINE__, cudaGetErrorString(err));          \
            exit(EXIT_FAILURE);                                            \
        }                                                                  \
    } while (0)

// A simple kernel
__global__ void safeKernel(int *data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] *= 2;
    }
}

int main() {
    printf("=== CUDA Error Checking Demo ===\n\n");
    
    int n = 1000;
    size_t size = n * sizeof(int);
    
    // Host allocation (regular C)
    int *h_data = (int*)malloc(size);
    for (int i = 0; i < n; i++) h_data[i] = i;
    
    // Device allocation WITH error checking
    int *d_data;
    CHECK_CUDA(cudaMalloc(&d_data, size));
    printf("✅ cudaMalloc successful\n");
    
    // Memory copy WITH error checking
    CHECK_CUDA(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice));
    printf("✅ cudaMemcpy (H→D) successful\n");
    
    // Kernel launch (cannot wrap in CHECK_CUDA directly)
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    
    safeKernel<<<blocksPerGrid, threadsPerBlock>>>(d_data, n);
    CHECK_KERNEL();  // Check for launch errors
    
    CHECK_CUDA(cudaDeviceSynchronize());  // Check for execution errors
    printf("✅ Kernel execution successful\n");
    
    // Copy back WITH error checking
    CHECK_CUDA(cudaMemcpy(h_data, d_data, size, cudaMemcpyDeviceToHost));
    printf("✅ cudaMemcpy (D→H) successful\n");
    
    // Verify
    bool correct = true;
    for (int i = 0; i < n; i++) {
        if (h_data[i] != i * 2) { correct = false; break; }
    }
    printf("\nResult: %s\n", correct ? "✅ CORRECT" : "❌ WRONG");
    
    // Cleanup
    CHECK_CUDA(cudaFree(d_data));
    free(h_data);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 error_checking.cu -o error_checking && ./error_checking

### Common CUDA Errors

Let's intentionally trigger some common errors to see how they appear:

In [ ]:
%%writefile common_errors.cu
/**
 * Common CUDA Errors and How to Catch Them
 */

#include <stdio.h>
#include <cuda_runtime.h>

#define CHECK_CUDA(call)                                                    \
    do {                                                                    \
        cudaError_t err = call;                                            \
        if (err != cudaSuccess) {                                          \
            printf("❌ CUDA Error: %s\n", cudaGetErrorString(err));        \
            return;                                                        \
        }                                                                  \
    } while (0)

__global__ void simpleKernel() {
    // Does nothing - just for testing launch configs
}

void testInvalidConfiguration() {
    printf("\n=== Test 1: Invalid Launch Configuration ===\n");
    
    // Error: More than 1024 threads per block
    simpleKernel<<<1, 2048>>>();  // Max is typically 1024!
    
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("❌ Caught error: %s\n", cudaGetErrorString(err));
    }
    cudaDeviceSynchronize();
}

void testOutOfMemory() {
    printf("\n=== Test 2: Out of Memory ===\n");
    
    // Try to allocate 1 TB of GPU memory
    void *ptr;
    cudaError_t err = cudaMalloc(&ptr, 1024ULL * 1024 * 1024 * 1024);
    
    if (err != cudaSuccess) {
        printf("❌ Caught error: %s\n", cudaGetErrorString(err));
    }
}

void testInvalidDevice() {
    printf("\n=== Test 3: Invalid Device ===\n");
    
    // Try to use device 999
    cudaError_t err = cudaSetDevice(999);
    
    if (err != cudaSuccess) {
        printf("❌ Caught error: %s\n", cudaGetErrorString(err));
    }
    
    // Reset to device 0
    cudaSetDevice(0);
}

int main() {
    printf("=== Common CUDA Errors Demo ===\n");
    
    testInvalidConfiguration();
    testOutOfMemory();
    testInvalidDevice();
    
    printf("\n✅ All error tests completed!\n");
    printf("   Key lesson: ALWAYS check CUDA return values!\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 common_errors.cu -o common_errors && ./common_errors

### Using compute-sanitizer (Memory Checker)

The `compute-sanitizer` tool (replaces cuda-memcheck) detects memory errors at runtime:

In [ ]:
%%writefile memory_bug.cu
/**
 * A program with an intentional out-of-bounds memory access
 * Use compute-sanitizer to detect it!
 */

#include <stdio.h>
#include <cuda_runtime.h>

__global__ void buggyKernel(int *data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    // BUG: No bounds check! Will access invalid memory!
    data[idx] = idx;  // idx can exceed n!
}

int main() {
    int n = 100;
    int *d_data;
    cudaMalloc(&d_data, n * sizeof(int));
    
    // Launch with MORE threads than array elements
    // This will cause out-of-bounds writes
    buggyKernel<<<1, 256>>>(d_data, n);  // 256 threads, but only 100 elements!
    cudaDeviceSynchronize();
    
    cudaFree(d_data);
    printf("Program completed (but had memory errors!)\n");
    return 0;
}

In [ ]:
# Compile with debug info for better error messages
!nvcc -arch=sm_75 -g -G memory_bug.cu -o memory_bug

# Run with compute-sanitizer (if available)
!which compute-sanitizer && compute-sanitizer ./memory_bug || echo "compute-sanitizer not available in Colab, but would detect: Invalid __global__ write"

### CUDA C++ Error Handling Summary

| Pattern | When to Use |
|---------|-------------|
| `CHECK_CUDA(call)` | Wrap all CUDA API calls |
| `CHECK_KERNEL()` | After kernel launches |
| `cudaGetLastError()` | Get most recent error |
| `cudaGetErrorString(err)` | Convert error to text |
| `cudaDeviceSynchronize()` | Force sync and catch async errors |
| `compute-sanitizer` | Detect memory errors at runtime |

**Essential Error Checking Rules:**
1. **Always** check return values of CUDA API calls
2. **Always** call `cudaGetLastError()` after kernel launches
3. **Always** sync before checking results
4. Use debug builds (`-g -G`) during development
5. Test with `compute-sanitizer` regularly

---

## Part 2: Python/Numba (Optional Comparison)

The following sections demonstrate error handling using Python and Numba.

In [ ]:
# ⚙️ Colab Setup Cell - Run this first!
import subprocess, sys
try:
    import google.colab
    print("🔧 Running on Google Colab - Installing dependencies...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
    print("✅ Setup complete!")
except ImportError:
    print("💻 Running locally - make sure you have: pip install numba numpy")

# Day 4: Error Handling & Debugging

Bugs in CUDA code can be subtle and hard to find. Today you'll learn:
- How CUDA errors work
- Proper error checking patterns
- Common pitfalls and how to avoid them
- Debugging techniques
- Using CUDA-MEMCHECK and compute-sanitizer

---

## 1. Understanding CUDA Errors

CUDA operations can fail for many reasons:
- Invalid kernel launch configuration
- Out of memory
- Device not available
- Invalid memory access
- Race conditions

**Key concept:** CUDA operations are often **asynchronous**. Errors may not be reported until later!

```
kernel<<<grid, block>>>(...);  // Launches, returns immediately
// ... other code ...
cudaDeviceSynchronize();       // Error might appear HERE!
```

In [ ]:
# Setup
import numpy as np
from numba import cuda
from numba.cuda.cudadrv.driver import CudaAPIError
import math
import traceback

print("CUDA device:", cuda.get_current_device().name.decode())

## 2. Common CUDA Errors & How to Trigger Them

Let's intentionally cause errors to understand how they appear.

In [ ]:
# Error 1: Invalid Launch Configuration
# Max threads per block is 1024, what happens if we exceed it?

@cuda.jit
def simple_kernel(arr):
    idx = cuda.grid(1)
    if idx < arr.size:
        arr[idx] = idx

arr = np.zeros(100, dtype=np.float32)
arr_d = cuda.to_device(arr)

print("Attempting to launch with 2048 threads per block...")
print("(Max allowed is 1024)")
print("-" * 50)

try:
    # This will fail - too many threads per block!
    simple_kernel[1, 2048](arr_d)
    cuda.synchronize()
except Exception as e:
    print(f"❌ Error caught: {type(e).__name__}")
    print(f"   Message: {e}")

In [ ]:
# Error 2: Out of Memory
# Trying to allocate more than available GPU memory

ctx = cuda.current_context()
free_mem, total_mem = ctx.get_memory_info()
print(f"Free GPU memory: {free_mem / 1e9:.2f} GB")
print(f"Attempting to allocate: {free_mem * 2 / 1e9:.2f} GB (2x available)")
print("-" * 50)

try:
    # Try to allocate more than available
    huge_array = cuda.device_array(int(free_mem * 2), dtype=np.uint8)
except Exception as e:
    print(f"❌ Error caught: {type(e).__name__}")
    print(f"   Message: {e}")

## 3. The Debugging Checklist

When your CUDA code doesn't work, check these in order:

### 🔍 Checklist

1. **Is CUDA available?**
   ```python
   cuda.is_available()
   ```

2. **Are launch parameters valid?**
   - `threads_per_block` ≤ 1024
   - `blocks` > 0
   - Grid dimensions within limits

3. **Is there enough memory?**
   - Check `cuda.current_context().get_memory_info()`

4. **Are array sizes correct?**
   - Boundary checks in kernel: `if idx < n:`

5. **Are data types matching?**
   - GPU prefers float32, not float64

6. **Did you synchronize?**
   - `cuda.synchronize()` before reading results

In [ ]:
# Helper function: Safe kernel launch wrapper
def safe_launch(kernel, grid, block, *args, **kwargs):
    """Launch kernel with error checking"""
    device = cuda.get_current_device()
    
    # Validate block size
    if isinstance(block, int):
        block = (block,)
    total_threads = 1
    for dim in block:
        total_threads *= dim
    if total_threads > device.MAX_THREADS_PER_BLOCK:
        raise ValueError(f"Block size {block} = {total_threads} threads exceeds max {device.MAX_THREADS_PER_BLOCK}")
    
    # Validate grid size
    if isinstance(grid, int):
        grid = (grid,)
    for i, dim in enumerate(grid):
        max_dim = [device.MAX_GRID_DIM_X, device.MAX_GRID_DIM_Y, device.MAX_GRID_DIM_Z][i]
        if dim > max_dim:
            raise ValueError(f"Grid dimension {i} = {dim} exceeds max {max_dim}")
    
    # Launch
    kernel[grid, block](*args, **kwargs)
    cuda.synchronize()

# Test safe launch
print("Testing safe_launch helper:")
arr = cuda.device_array(100, dtype=np.float32)

try:
    safe_launch(simple_kernel, 1, 2048, arr)  # Should fail validation
except ValueError as e:
    print(f"✅ Caught before launch: {e}")

safe_launch(simple_kernel, 1, 256, arr)  # Should work
print("✅ Valid launch succeeded")

## 4. Common Pitfalls & Bug Patterns

### Pitfall 1: Missing Boundary Check

In [ ]:
# BAD: No boundary check
@cuda.jit
def bad_kernel_no_bounds(arr):
    idx = cuda.grid(1)
    arr[idx] = idx  # 💥 Will access out-of-bounds memory!

# GOOD: With boundary check
@cuda.jit  
def good_kernel_with_bounds(arr, n):
    idx = cuda.grid(1)
    if idx < n:  # ✅ Always check!
        arr[idx] = idx

# Demonstrate the difference
n = 100
arr = cuda.device_array(n, dtype=np.float32)
threads = 256  # More threads than elements!
blocks = 1

print("With proper bounds checking:")
good_kernel_with_bounds[blocks, threads](arr, n)
cuda.synchronize()
print("✅ Completed safely")

### Pitfall 2: Wrong Data Type

In [ ]:
# NumPy defaults to float64, but CUDA prefers float32
import time

@cuda.jit
def add_arrays(a, b, c):
    idx = cuda.grid(1)
    if idx < c.size:
        c[idx] = a[idx] + b[idx]

n = 10_000_000

# float64 (default) - slower on most GPUs
a64 = np.random.randn(n)  # Default is float64!
b64 = np.random.randn(n)
c64 = np.zeros(n)

# float32 - preferred
a32 = np.random.randn(n).astype(np.float32)
b32 = np.random.randn(n).astype(np.float32)
c32 = np.zeros(n, dtype=np.float32)

threads, blocks = 256, math.ceil(n / 256)

# Benchmark float64
a64_d, b64_d = cuda.to_device(a64), cuda.to_device(b64)
c64_d = cuda.device_array(n, dtype=np.float64)
add_arrays[blocks, threads](a64_d, b64_d, c64_d)
cuda.synchronize()

start = time.perf_counter()
for _ in range(10):
    add_arrays[blocks, threads](a64_d, b64_d, c64_d)
cuda.synchronize()
time64 = (time.perf_counter() - start) / 10

# Benchmark float32
a32_d, b32_d = cuda.to_device(a32), cuda.to_device(b32)
c32_d = cuda.device_array(n, dtype=np.float32)

start = time.perf_counter()
for _ in range(10):
    add_arrays[blocks, threads](a32_d, b32_d, c32_d)
cuda.synchronize()
time32 = (time.perf_counter() - start) / 10

print(f"float64: {time64*1000:.3f} ms")
print(f"float32: {time32*1000:.3f} ms")
print(f"Speedup: {time64/time32:.2f}x")
print("\n💡 Tip: Always use .astype(np.float32) unless you need float64 precision!")

### Pitfall 3: Forgetting to Synchronize

In [ ]:
# Kernel execution is ASYNCHRONOUS
@cuda.jit
def slow_kernel(arr):
    """Simulate slow computation"""
    idx = cuda.grid(1)
    if idx < arr.size:
        # Busy work
        val = 0.0
        for i in range(1000):
            val += idx * 0.001
        arr[idx] = val

arr = cuda.device_array(10000, dtype=np.float32)
threads, blocks = 256, math.ceil(10000 / 256)

# BAD: Timing without synchronization
start = time.perf_counter()
slow_kernel[blocks, threads](arr)
# Missing: cuda.synchronize()
bad_time = time.perf_counter() - start
print(f"Without sync: {bad_time*1000:.3f} ms (WRONG! Kernel still running)")

# GOOD: Proper timing with synchronization  
start = time.perf_counter()
slow_kernel[blocks, threads](arr)
cuda.synchronize()  # Wait for kernel to complete
good_time = time.perf_counter() - start
print(f"With sync:    {good_time*1000:.3f} ms (Correct)")

print(f"\n⚠️ The unsynchronized time is {good_time/bad_time:.0f}x too fast!")

## 5. Debugging with Print Statements

In Numba CUDA, you can use `print()` inside kernels for debugging (but use sparingly - it's slow!).

In [ ]:
@cuda.jit
def debug_kernel(arr, n):
    idx = cuda.grid(1)
    
    # Only print from first few threads to avoid output flood
    if idx < 3:
        print("Thread", idx, "starting")
    
    if idx < n:
        arr[idx] = idx * 2
        
        # Debug: Print values for first few elements
        if idx < 3:
            print("Thread", idx, "wrote value", arr[idx])

# Run with small array
arr = cuda.device_array(10, dtype=np.float32)
debug_kernel[1, 10](arr, 10)
cuda.synchronize()

print("\nFinal array:", arr.copy_to_host())

## 🎯 Exercises

### Exercise 1: Error-Proof Kernel Wrapper
Create a robust wrapper function that validates all inputs before launching a kernel.

In [ ]:
# TODO Exercise 1: Complete this error-proof wrapper

def launch_kernel_safe(kernel, data, threads_per_block=256):
    """
    Safely launch a kernel with automatic configuration and error checking.
    
    Args:
        kernel: The CUDA kernel function
        data: Input array (numpy or device array)
        threads_per_block: Threads per block (default 256)
    
    Returns:
        Device array with results
        
    Raises:
        ValueError: If inputs are invalid
        MemoryError: If not enough GPU memory
    """
    # TODO: Implement the following checks:
    # 1. Verify CUDA is available
    # 2. Check data is not empty
    # 3. Validate threads_per_block (1-1024)
    # 4. Check sufficient GPU memory
    # 5. Launch kernel with proper grid configuration
    # 6. Synchronize and check for errors
    
    pass

# Test your implementation
# ...

## 📝 Key Takeaways

### Error Handling Best Practices:

1. **Always synchronize** before reading results or timing
   ```python
   kernel[grid, block](...)
   cuda.synchronize()  # Wait for completion
   result = output.copy_to_host()
   ```

2. **Validate launch configuration**
   - threads_per_block ≤ 1024
   - Check grid dimensions against device limits

3. **Always include boundary checks**
   ```python
   if idx < n:
       arr[idx] = ...
   ```

4. **Use try/except for error handling**
   ```python
   try:
       kernel[grid, block](...)
   except CudaAPIError as e:
       print(f"CUDA Error: {e}")
   ```

5. **Prefer float32** unless you need float64 precision

6. **Debug strategically**
   - Use print() sparingly (only first few threads)
   - Use small test cases first
   - Verify CPU results before GPU

---

### 📚 Next Steps
You've completed Week 1! Before moving on:
1. Complete the checkpoint quiz
2. Finish all exercises in each notebook
3. Make sure you can run all code without errors

### 🔗 Resources
- [Error Handling Guide](../../cuda-programming-guide/02-basics/nvcc.md)
- [Debugging Documentation](../../cuda-programming-guide/04-special-topics/error-log-management.md)